In [6]:
import json

from pathlib import Path
from pprint import pprint

import toolz.curried as curried
from toolz.functoolz import pipe
from toolz.itertoolz import groupby


In [7]:
BASE_DIRECTORY = Path('./PerformanceHistory/Programs/json')

SEASONS = ['1960-61', '1961-62', '1962-63', '1963-64', '1964-65', '1965-66', 
           '1966-67', '1967-68', '1968-69', '1969-70', '1970-71', '1971-72', 
           '1972-73', '1973-74', '1974-75', '1975-76', '1976-77', '1977-78', 
           '1978-79', '1979-80', '1980-81']

In [8]:
def has_subscription_concert(program):
    for concert in program['concerts']:
        if concert['eventType'] == 'Subscription Season':
            return True
    return False

In [9]:
data_files = pipe(BASE_DIRECTORY.iterdir(),
                  curried.filter(lambda p: p.suffix == ".json"),
                  sorted)

for i, p in enumerate(data_files):
    print(f'{i:>3} {p.name}')

  0 1842-43_TO_1910-11.json
  1 1911-12_TO_1919-20.json
  2 1920-21_TO_1925-26.json
  3 1926-27_TO_1931-32.json
  4 1932-33_TO_1939-40.json
  5 1940-41_TO_1946-47.json
  6 1947-48_TO_1954-55.json
  7 1955-56_TO_1962-63.json
  8 1963-64_TO_1973-74.json
  9 1974-75_TO_1983-84.json
 10 1984-85_TO_1994-95.json
 11 1995-96_TO_2003-04.json
 12 2004-05_TO_2010-11.json
 13 2011-12_TO_NOW.json
 14 complete.json


In [12]:
all_programs = []
for i in range(0, 14):
    
    with data_files[i].open() as fp:
        print(f'Processing file {i:>2} "{data_files[i]}"')
        root = json.load(fp)
        all_programs = all_programs + root['programs']

Processing file  0 "PerformanceHistory/Programs/json/1842-43_TO_1910-11.json"
Processing file  1 "PerformanceHistory/Programs/json/1911-12_TO_1919-20.json"
Processing file  2 "PerformanceHistory/Programs/json/1920-21_TO_1925-26.json"
Processing file  3 "PerformanceHistory/Programs/json/1926-27_TO_1931-32.json"
Processing file  4 "PerformanceHistory/Programs/json/1932-33_TO_1939-40.json"
Processing file  5 "PerformanceHistory/Programs/json/1940-41_TO_1946-47.json"
Processing file  6 "PerformanceHistory/Programs/json/1947-48_TO_1954-55.json"
Processing file  7 "PerformanceHistory/Programs/json/1955-56_TO_1962-63.json"
Processing file  8 "PerformanceHistory/Programs/json/1963-64_TO_1973-74.json"
Processing file  9 "PerformanceHistory/Programs/json/1974-75_TO_1983-84.json"
Processing file 10 "PerformanceHistory/Programs/json/1984-85_TO_1994-95.json"
Processing file 11 "PerformanceHistory/Programs/json/1995-96_TO_2003-04.json"
Processing file 12 "PerformanceHistory/Programs/json/2004-05_TO_

In [13]:
all_subscription_programs = [p for p in all_programs 
                             if has_subscription_concert(p)]

In [14]:
len(all_subscription_programs)

6611

In [15]:
with open('all_subscription_programs.json', mode='w') as fp:
    json.dump(all_subscription_programs, fp, indent=2)

In [103]:
# Now filter by season

all_programs_filtered = [p for p in all_programs if p['season'] in SEASONS]

In [157]:
with open('nyphil_subscription_programs_1960_1981.json', mode='w') as fp:
    json.dump(all_subscription_programs, fp, indent=2)

In [118]:
def has_works_by(program, composer_name):
    for work in program['works']:
        if 'composerName' in work and (work['composerName'].find(composer_name) > -1):
            return True
    return False

In [149]:
def get_programs_for_composer(composer_name):
    return [p for p in all_subscription_programs 
            if has_works_by(p, composer_name)]

def get_works_for_composer(program, composer_name):
    retval = {}
    for key in ['id', 'programID', 'season']:
        retval[key] = program[key]
    retval['works'] = {}
    for work in program['works']:
        if 'composer_name' in work and work['composer_name'].find(composer_name) > -1:
            retval['works'].append(work)
    return retval

In [156]:
programs = get_programs_for_composer('Stravinsky')
# programs_str = json.dumps(programs, indent=2)
print(len(programs))
print(programs_str)


70
[
  {
    "id": "7f1bc02f-1faa-4493-910e-78c70553c633-0.1",
    "programID": "1346",
    "orchestra": "New York Philharmonic",
    "season": "1960-61",
    "concerts": [
      {
        "eventType": "Subscription Season",
        "Location": "Manhattan, NY",
        "Venue": "Carnegie Hall",
        "Date": "1960-10-02T04:00:00Z",
        "Time": "3:00PM"
      }
    ],
    "works": [
      {
        "ID": "52389*",
        "composerName": "Beethoven,  Ludwig  van",
        "workTitle": "LEONORE OVERTURE NO. 3, OP. 72B",
        "conductorName": "Shapira, Elyakum",
        "soloists": []
      },
      {
        "ID": "53078*",
        "composerName": "Schumann,  Robert",
        "workTitle": "SYMPHONY NO. 4, D MINOR, OP. 120",
        "conductorName": "Millar, Gregory",
        "soloists": []
      },
      {
        "ID": "0*",
        "interval": "Intermission",
        "soloists": []
      },
      {
        "ID": "52644*",
        "composerName": "Debussy,  Claude",
        "wo

In [53]:
repeated_programs = [program for program in programs['programs'] if len(program['concerts']) > 1]

In [55]:
repeated_programs

[{'id': '53baad78-6929-4419-8f4f-6ac4204fcb2a-0.1',
  'programID': '7211',
  'orchestra': 'New York Philharmonic',
  'season': '1955-56',
  'concerts': [{'eventType': 'Subscription Season',
    'Location': 'Manhattan, NY',
    'Venue': 'Carnegie Hall',
    'Date': '1955-10-20T04:00:00Z',
    'Time': '8:45PM'},
   {'eventType': 'Subscription Season',
    'Location': 'Manhattan, NY',
    'Venue': 'Carnegie Hall',
    'Date': '1955-10-21T04:00:00Z',
    'Time': '2:30PM'}],
  'works': [{'ID': '3965*',
    'composerName': 'Anthem,',
    'workTitle': 'UNITED STATES',
    'conductorName': 'Mitropoulos, Dimitri',
    'soloists': []},
   {'ID': '1245*',
    'composerName': 'Mozart,  Wolfgang  Amadeus',
    'workTitle': 'ABDUCTION FROM THE SERAGLIO, THE: OVERTURE (ARR. Busoni)',
    'conductorName': 'Mitropoulos, Dimitri',
    'soloists': []},
   {'ID': '51071*',
    'composerName': 'Mozart,  Wolfgang  Amadeus',
    'workTitle': 'CONCERTO, PIANO NO. 25, C MAJOR, K.503',
    'conductorName': 'Mit

In [54]:
print(len(repeated_programs))

262


In [57]:
single_performance_subscription_concerts = [p for p in programs['programs'] 
                                            if len(p['concerts']) == 1
                                            and p['concerts'][0]['eventType'] == 'Subscription Season']
len(single_performance_subscription_concerts)

186

In [58]:
single_performance_subscription_concerts

[{'id': 'f26174c5-ef03-479b-867c-ed46149d27a9-0.1',
  'programID': '7217',
  'orchestra': 'New York Philharmonic',
  'season': '1955-56',
  'concerts': [{'eventType': 'Subscription Season',
    'Location': 'Manhattan, NY',
    'Venue': 'Carnegie Hall',
    'Date': '1955-10-23T04:00:00Z',
    'Time': '2:30PM'}],
  'works': [{'ID': '548*',
    'composerName': 'Mozart,  Wolfgang  Amadeus',
    'workTitle': 'SINFONIA CONCERTANTE, VLN/VLA/VC, A MAJOR, K.320E [ANH. 104]',
    'conductorName': 'Mitropoulos, Dimitri',
    'soloists': [{'soloistName': 'Corigliano, John, Sr.',
      'soloistInstrument': 'Violin',
      'soloistRoles': 'A'},
     {'soloistName': 'Lincer, William',
      'soloistInstrument': 'Viola',
      'soloistRoles': 'A'},
     {'soloistName': 'Varga, Laszlo',
      'soloistInstrument': 'Cello',
      'soloistRoles': 'A'}]},
   {'ID': '51041*',
    'composerName': 'Mozart,  Wolfgang  Amadeus',
    'workTitle': 'CONCERTO, PIANO NO. 16, D MAJOR, K.451',
    'conductorName': 'Mi

In [65]:
for program in all_subscription_programs:
    for concert in program['concerts']:
        if concert['eventType'] != 'Subscription Season':
            print(program)

In [81]:
walton_programs = [p for p in all_subscription_programs if has_works_by(p, 'Walton')]
walton_programs

[{'id': 'ede4acbb-2ad2-4483-b038-d867aa0bd471-0.1',
  'programID': '2786',
  'orchestra': 'New York Philharmonic',
  'season': '1956-57',
  'concerts': [{'eventType': 'Subscription Season',
    'Location': 'Manhattan, NY',
    'Venue': 'Carnegie Hall',
    'Date': '1957-02-09T05:00:00Z',
    'Time': '8:45PM'}],
  'works': [{'ID': '50840*1',
    'composerName': 'Wolf-Ferrari,  Ermanno',
    'workTitle': 'THE SECRET OF SUZANNE',
    'movement': 'Overture',
    'conductorName': 'Kostelanetz, Andre',
    'soloists': []},
   {'ID': '642*7',
    'composerName': 'Puccini,  Giacomo',
    'workTitle': 'BOHEME, LA',
    'movement': 'Orchestral Excerpts (unspecified) (Act I)',
    'conductorName': 'Kostelanetz, Andre',
    'soloists': []},
   {'ID': '51149*',
    'composerName': 'Liszt,  Franz',
    'workTitle': 'PRELUDES, LES (SYMPHONIC POEM NO. 3)',
    'conductorName': 'Kostelanetz, Andre',
    'soloists': []},
   {'ID': '0*', 'interval': 'Intermission', 'soloists': []},
   {'ID': '3635*',
   

In [89]:
from itertools import pairwise
import pyperclip
import io

seasons = [f'{y1}-{y2-1900}' for y1, y2 in pairwise(range(1960, 1982))]

with io.StringIO() as fp:
    fp.write("[")
    for season in seasons:
        fp.write(f"'{season}', ")
    s = fp.getvalue().strip()[:-1] + "]"
     
pyperclip.copy(s)

In [98]:
seasons = {p['season'] for p in all_subscription_programs}

In [99]:
seasons

{'1955-56',
 '1956-57',
 '1957-58',
 '1958-59',
 '1959-60',
 '1960-61',
 '1961-62',
 '1962-63'}

In [102]:
subscription_programs = {}
subscription_programs['1960-63'] = [p for p in all_subscription_programs
                                    if p['season'] in SEASONS]
len(subscription_programs['1960-63'])



123